In [1]:
import pandas as pd
import numpy as np
import xgboost as xg
import math

# Import  Data 

In [2]:
dates=pd.read_csv('C:\\Users\\MSI\\Desktop\\AI Hack Tunisia 2022\\AI Hack Tunisia 2022\\Grocery Store Product Forecasting Challenge\\dates.csv')
display(dates.head())
print(dates.shape)

,date,year,month,dayofmonth,dayofweek,dayofyear,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear
0,365,1,1,1,2,1,1,1,True,False,True,False,True,False,101
1,366,1,1,2,3,2,1,1,False,False,False,False,False,False,101
2,367,1,1,3,4,3,1,1,False,False,False,False,False,False,101
3,368,1,1,4,5,4,1,1,False,False,False,False,False,False,101
4,369,1,1,5,6,5,1,1,False,False,False,False,False,False,101


(1320, 15)


In [4]:
dates["date"]

0        365
1        366
2        367
3        368
4        369
        ... 
1315    1680
1316    1681
1317    1682
1318    1683
1319    1684
Name: date, Length: 1320, dtype: int64

In [5]:
holidays=pd.read_csv('C:\\Users\\MSI\\Desktop\\AI Hack Tunisia 2022\\AI Hack Tunisia 2022\\Grocery Store Product Forecasting Challenge\\holidays.csv')
display(holidays.head())
print(holidays.shape)

,date,type
0,1,0
1,5,4
2,12,4
3,42,0
4,43,0


(281, 2)


In [6]:
holidays["date"]

0         1
1         5
2        12
3        42
4        43
       ... 
276    1663
277    1674
278    1679
279    1680
280    1684
Name: date, Length: 281, dtype: int64

In [7]:
stores=pd.read_csv('C:\\Users\\MSI\\Desktop\\AI Hack Tunisia 2022\\AI Hack Tunisia 2022\\Grocery Store Product Forecasting Challenge\\stores.csv')
display(stores.head())
print(stores.shape)

,store_id,city,type,cluster
0,store_1,0,0,0
1,store_2,0,0,0
2,store_3,0,0,1
3,store_4,0,0,2
4,store_5,1,0,3


(54, 4)


In [8]:
train=pd.read_csv('C:\\Users\\MSI\\Desktop\\AI Hack Tunisia 2022\\AI Hack Tunisia 2022\\Grocery Store Product Forecasting Challenge\\train.csv')
display(train.head())
print(train.shape)

,date,store_id,category_id,target,onpromotion,nbr_of_transactions
0,365,store_1,category_24,0.0,0,0.0
1,365,store_1,category_21,0.0,0,0.0
2,365,store_1,category_32,0.0,0,0.0
3,365,store_1,category_18,0.0,0,0.0
4,365,store_1,category_26,0.0,0,0.0


(2248884, 6)


In [9]:
train["date"]


0           365
1           365
2           365
3           365
4           365
           ... 
2248879    1626
2248880    1626
2248881    1626
2248882    1626
2248883    1626
Name: date, Length: 2248884, dtype: int64

In [10]:
a=np.array(train["date"])
a

array([ 365,  365,  365, ..., 1626, 1626, 1626], dtype=int64)

In [11]:
a.sort()
a

array([ 365,  365,  365, ..., 1626, 1626, 1626], dtype=int64)

In [12]:
test=pd.read_csv('C:\\Users\\MSI\\Desktop\\AI Hack Tunisia 2022\\AI Hack Tunisia 2022\\Grocery Store Product Forecasting Challenge\\test.csv')
display(test.head())
print(test.shape)

,date,store_id,category_id,onpromotion
0,1627,store_1,category_24,0
1,1627,store_1,category_21,0
2,1627,store_1,category_32,0
3,1627,store_1,category_18,16
4,1627,store_1,category_26,0


(99792, 4)


In [13]:
len(test["date"].unique())

56

In [14]:
SampleSubmission=pd.read_csv('C:\\Users\\MSI\\Desktop\\AI Hack Tunisia 2022\\AI Hack Tunisia 2022\\Grocery Store Product Forecasting Challenge\\SampleSubmission.csv')
display(SampleSubmission.head())
print(SampleSubmission.shape)

,ID,target
0,year_week_425_store_1_category_0,0
1,year_week_426_store_1_category_0,0
2,year_week_427_store_1_category_0,1
3,year_week_428_store_1_category_0,3
4,year_week_429_store_1_category_0,2


(14256, 2)


# Merging Train and other tables

In [15]:
train=train.merge(dates)
train.head()

,date,store_id,category_id,target,onpromotion,nbr_of_transactions,year,month,dayofmonth,dayofweek,dayofyear,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear
0,365,store_1,category_24,0.0,0,0.0,1,1,1,2,1,1,1,True,False,True,False,True,False,101
1,365,store_1,category_21,0.0,0,0.0,1,1,1,2,1,1,1,True,False,True,False,True,False,101
2,365,store_1,category_32,0.0,0,0.0,1,1,1,2,1,1,1,True,False,True,False,True,False,101
3,365,store_1,category_18,0.0,0,0.0,1,1,1,2,1,1,1,True,False,True,False,True,False,101
4,365,store_1,category_26,0.0,0,0.0,1,1,1,2,1,1,1,True,False,True,False,True,False,101


In [16]:
train=train.merge(stores)
train.head()

,date,store_id,category_id,target,onpromotion,nbr_of_transactions,year,month,dayofmonth,dayofweek,...,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear,city,type,cluster
0,365,store_1,category_24,0.0,0,0.0,1,1,1,2,...,True,False,True,False,True,False,101,0,0,0
1,365,store_1,category_21,0.0,0,0.0,1,1,1,2,...,True,False,True,False,True,False,101,0,0,0
2,365,store_1,category_32,0.0,0,0.0,1,1,1,2,...,True,False,True,False,True,False,101,0,0,0
3,365,store_1,category_18,0.0,0,0.0,1,1,1,2,...,True,False,True,False,True,False,101,0,0,0
4,365,store_1,category_26,0.0,0,0.0,1,1,1,2,...,True,False,True,False,True,False,101,0,0,0


In [17]:
train=train.drop(["year","month","dayofmonth","dayofweek","dayofyear","weekofyear","quarter","is_month_start","is_month_end"
                  ,"is_quarter_start","is_quarter_end","is_year_start","is_year_end"],axis=1)
train.head()

,date,store_id,category_id,target,onpromotion,nbr_of_transactions,year_weekofyear,city,type,cluster
0,365,store_1,category_24,0.0,0,0.0,101,0,0,0
1,365,store_1,category_21,0.0,0,0.0,101,0,0,0
2,365,store_1,category_32,0.0,0,0.0,101,0,0,0
3,365,store_1,category_18,0.0,0,0.0,101,0,0,0
4,365,store_1,category_26,0.0,0,0.0,101,0,0,0


In [18]:
train["ID"] ="year_week"+"_"+train['year_weekofyear'].astype(str) +"_"+ train["store_id"]+"_"+train["category_id"]
train

,date,store_id,category_id,target,onpromotion,nbr_of_transactions,year_weekofyear,city,type,cluster,ID
0,365,store_1,category_24,0.000,0,0.0,101,0,0,0,year_week_101_store_1_category_24
1,365,store_1,category_21,0.000,0,0.0,101,0,0,0,year_week_101_store_1_category_21
2,365,store_1,category_32,0.000,0,0.0,101,0,0,0,year_week_101_store_1_category_32
3,365,store_1,category_18,0.000,0,0.0,101,0,0,0,year_week_101_store_1_category_18
4,365,store_1,category_26,0.000,0,0.0,101,0,0,0,year_week_101_store_1_category_26
...,...,...,...,...,...,...,...,...,...,...,...
2248879,1626,store_9,category_23,539.725,0,2141.0,424,0,1,4,year_week_424_store_9_category_23
2248880,1626,store_9,category_20,84.177,0,2141.0,424,0,1,4,year_week_424_store_9_category_20
2248881,1626,store_9,category_15,1973.760,3,2141.0,424,0,1,4,year_week_424_store_9_category_15
2248882,1626,store_9,category_29,2.000,0,2141.0,424,0,1,4,year_week_424_store_9_category_29


# Merging Test and other tables

In [19]:
test=test.merge(dates)
test.head()

,date,store_id,category_id,onpromotion,year,month,dayofmonth,dayofweek,dayofyear,weekofyear,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear
0,1627,store_1,category_24,0,4,6,19,0,170,25,2,False,False,False,False,False,False,425
1,1627,store_1,category_21,0,4,6,19,0,170,25,2,False,False,False,False,False,False,425
2,1627,store_1,category_32,0,4,6,19,0,170,25,2,False,False,False,False,False,False,425
3,1627,store_1,category_18,16,4,6,19,0,170,25,2,False,False,False,False,False,False,425
4,1627,store_1,category_26,0,4,6,19,0,170,25,2,False,False,False,False,False,False,425


In [20]:
test=test.merge(stores)
test.head()

,date,store_id,category_id,onpromotion,year,month,dayofmonth,dayofweek,dayofyear,weekofyear,...,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear,city,type,cluster
0,1627,store_1,category_24,0,4,6,19,0,170,25,...,False,False,False,False,False,False,425,0,0,0
1,1627,store_1,category_21,0,4,6,19,0,170,25,...,False,False,False,False,False,False,425,0,0,0
2,1627,store_1,category_32,0,4,6,19,0,170,25,...,False,False,False,False,False,False,425,0,0,0
3,1627,store_1,category_18,16,4,6,19,0,170,25,...,False,False,False,False,False,False,425,0,0,0
4,1627,store_1,category_26,0,4,6,19,0,170,25,...,False,False,False,False,False,False,425,0,0,0


In [21]:
test=test.drop(["year","month","dayofmonth","dayofweek","dayofyear","weekofyear","quarter","is_month_start","is_month_end"
                  ,"is_quarter_start","is_quarter_end","is_year_start","is_year_end"],axis=1)
test.head()

,date,store_id,category_id,onpromotion,year_weekofyear,city,type,cluster
0,1627,store_1,category_24,0,425,0,0,0
1,1627,store_1,category_21,0,425,0,0,0
2,1627,store_1,category_32,0,425,0,0,0
3,1627,store_1,category_18,16,425,0,0,0
4,1627,store_1,category_26,0,425,0,0,0


In [22]:
test["ID"] ="year_week"+"_"+test['year_weekofyear'].astype(str) +"_"+ test["store_id"]+"_"+test["category_id"]
test

,date,store_id,category_id,onpromotion,year_weekofyear,city,type,cluster,ID
0,1627,store_1,category_24,0,425,0,0,0,year_week_425_store_1_category_24
1,1627,store_1,category_21,0,425,0,0,0,year_week_425_store_1_category_21
2,1627,store_1,category_32,0,425,0,0,0,year_week_425_store_1_category_32
3,1627,store_1,category_18,16,425,0,0,0,year_week_425_store_1_category_18
4,1627,store_1,category_26,0,425,0,0,0,year_week_425_store_1_category_26
...,...,...,...,...,...,...,...,...,...
99787,1682,store_9,category_23,0,432,0,1,4,year_week_432_store_9_category_23
99788,1682,store_9,category_20,1,432,0,1,4,year_week_432_store_9_category_20
99789,1682,store_9,category_15,7,432,0,1,4,year_week_432_store_9_category_15
99790,1682,store_9,category_29,8,432,0,1,4,year_week_432_store_9_category_29


# Transform categorical values : Train DataFrame

In [23]:
df1_train=pd.get_dummies(train["store_id"])

In [24]:
df2_train=pd.get_dummies(train["category_id"])

In [25]:
train=train.drop(["store_id","category_id"],axis=1)

In [26]:
train=train.join(df1_train)

In [27]:
train=train.join(df2_train)

In [28]:
train=train.drop("nbr_of_transactions",axis=1)
train.head()

,date,target,onpromotion,year_weekofyear,city,type,cluster,ID,store_1,store_10,...,category_3,category_30,category_31,category_32,category_4,category_5,category_6,category_7,category_8,category_9
0,365,0.0,0,101,0,0,0,year_week_101_store_1_category_24,1,0,...,0,0,0,0,0,0,0,0,0,0
1,365,0.0,0,101,0,0,0,year_week_101_store_1_category_21,1,0,...,0,0,0,0,0,0,0,0,0,0
2,365,0.0,0,101,0,0,0,year_week_101_store_1_category_32,1,0,...,0,0,0,1,0,0,0,0,0,0
3,365,0.0,0,101,0,0,0,year_week_101_store_1_category_18,1,0,...,0,0,0,0,0,0,0,0,0,0
4,365,0.0,0,101,0,0,0,year_week_101_store_1_category_26,1,0,...,0,0,0,0,0,0,0,0,0,0


# Transform categorical values : Test DataFrame

In [29]:
df1_test=pd.get_dummies(test["store_id"])

In [30]:
df2_test=pd.get_dummies(test["category_id"])

In [31]:
test=test.drop(["store_id","category_id"],axis=1)

In [32]:
test=test.join(df1_test)

In [33]:
test=test.join(df2_test)
test.head()

,date,onpromotion,year_weekofyear,city,type,cluster,ID,store_1,store_10,store_11,...,category_3,category_30,category_31,category_32,category_4,category_5,category_6,category_7,category_8,category_9
0,1627,0,425,0,0,0,year_week_425_store_1_category_24,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1627,0,425,0,0,0,year_week_425_store_1_category_21,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1627,0,425,0,0,0,year_week_425_store_1_category_32,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1627,16,425,0,0,0,year_week_425_store_1_category_18,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1627,0,425,0,0,0,year_week_425_store_1_category_26,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Model

In [39]:
train_X=train.drop(["target","ID"],axis=1)

In [40]:
train_y=train["target"]

In [ ]:
xgb_r = xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 13)

In [ ]:
xgb_r.fit(train_X, train_y)

In [ ]:
test_X=test

In [ ]:
test_X=test_X.drop("ID",axis=1)

In [ ]:
pred = xgb_r.predict(test_X)

In [ ]:
result=[]
for x in pred:
    result.append(round(math.log1p(x)))

In [ ]:
result

# Solution

In [ ]:
sub = pd.DataFrame({'ID': test.ID,
                   'target': result})

In [ ]:
output = pd.DataFrame({'ID': sub.ID, 'target': sub.target})

In [ ]:
output.to_csv('Bureau\\testing.csv', index=False)

In [36]:
from sklearn.model_selection import GridSearchCV

In [ ]:
PARAMETERS = {"subsample":[0.5, 0.75, 1],
              "colsample_bytree":[0.5, 0.75, 1],
              "max_depth":[2, 6, 12],
              "min_child_weight":[1,5,15],
              "learning_rate":[0.3, 0.1, 0.03],
              "n_estimators":[100]}



In [ ]:
xgb1 = xg.XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(train_X,
         train_y)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
